In [ ]:
using Plots
using QuadGK
using NLopt
using SpecialFunctions
using NLsolve
using Statistics
include("../../scaling_CGS.jl")

In [ ]:
#---------- Two Steam Setup ----------
base = BaseUnits("CGS")

electron_mass_scaling = 1e0
charge_scaling = 1e0
eps_0_scaling = 1e0
mu_0_scaling = 1e0

scale_base_units(base, eps_0_scaling, mu_0_scaling, charge_scaling, electron_mass_scaling)

mass_density_scale = 1.673532836356e-14
#length_scale       = 227.71078654314857#ion skin depth
length_scale       = 5.314093711399421 # electron skin depth
time_scale         = 1.1137516851221212e-9



#length= 227.71078654314857 #ion skin depth
length = 5.314093711399421 # electron skin depth
number_density=1e10
B_flux=10
#temperature   = 2.3719594179263036e6  # Corresponds to v_t_e = 0.1*V_0  for V_0 = 0.2*c 
temperature   = 2.3719594179263036e6 * 2.0^2

temperature_e = temperature
temperature_p = temperature



scaling = ScalingCGS(base,
                number_density, length, temperature, temperature_e, temperature_p, B_flux,
                length_scale, mass_density_scale, time_scale)

ds = 0.0025
per_cell = 125

set_macro_particle_weights(scaling, ds, per_cell);
print_all_CGS(scaling)

In [ ]:
c = scaling.c_real


m_i = scaling.m_p_real
m_e = scaling.m_e_real


t = scaling.temperature_real


n_i = scaling.number_density_real
n_e = scaling.number_density_real

k_B = scaling.k_B_real

V_0 = scaling.c_real * 0.2

v_t_e = scaling.v_thermal_e_real
v_t_p = scaling.v_thermal_p_real


omega_e = scaling.electron_plasma_freq_real
omega_p = scaling.proton_plasma_freq_real

debye_e = scaling.debye_len_real
debye_i = scaling.v_thermal_p_real / scaling.proton_plasma_freq_real


skin_depth = scaling.skin_depth_p_real

println(v_t_e / V_0)

In [ ]:
scaling.skin_depth_real

In [ ]:
c = scaling.c_code


m_i = scaling.m_p_code
m_e = scaling.m_e_code


t = scaling.temperature_code


n_i = scaling.number_density_code
n_e = scaling.number_density_code

k_B = scaling.k_B_code

V_0 = scaling.c_code * 0.2

v_t_e = scaling.v_thermal_e_code
v_t_p = scaling.v_thermal_p_code


omega_e = scaling.electron_plasma_freq_code
omega_p = scaling.proton_plasma_freq_code

debye_e = scaling.debye_len_code
debye_i = scaling.v_thermal_p_code / scaling.proton_plasma_freq_code


skin_depth = scaling.skin_depth_p_code

println(v_t_e / V_0)

In [ ]:
println(omega_e^2)


n_e_test = n_e * 0.3
n_e_test2 = n_e * 0.7
omega_e_test = scaling.e_code * sqrt(4 * pi * n_e_test / m_e)
omega_e_test2 = scaling.e_code * sqrt(4 * pi * n_e_test2 / m_e)

println(omega_e_test^2 + omega_e_test2^2)

In [ ]:
function W_val(z)
    #TODO Add special cases for small and large z???
    return 1. + 1im * z * sqrt(pi) * faddeeva(z)
end

#------- to verify this is implemented correctly --------------------
z_vals  = collect(range(-5,stop=5, length=1000))  #.+ 1.0im
W_res = W_val.(z_vals);

plot(real.(z_vals), real.(W_res),label="real")
plot!(real.(z_vals), imag.(W_res),label="imag")
#---------------------------------------------------------------


In [ ]:
#TODO we may rewrite all the eqautions so that omega_e refers to the frequency of each beam and not the total plasma frequency
#   This will allow us to write all the equations without the factor "1/2" in front of the electron terms
#   In general, it will allow us to built general two-stream dispersion relations

"
    full_dispertion_relation

compute the full dispersion relation for the warm electron-proton case
"
function full_dispertion_relation(omega, k, V0, v_t_e, v_t_i, debye_e, debye_i)
    z_plus  = (omega / k + V0) / (sqrt(2.) * v_t_e)
    z_minus = (omega / k - V0) / (sqrt(2.) * v_t_e)
    z_0     = (omega / k )     / (sqrt(2.) * v_t_i)
    
    electorn_part_1 = 1.0 / ( debye_e * k )^2.0 * 1.0/2.0 * W_val(z_plus)
    electorn_part_2 = 1.0 / ( debye_e * k )^2.0 * 1.0/2.0 * W_val(z_minus)
    ion_part        = 1.0 / ( debye_i * k )^2.0           * W_val(z_0)
    #TODO Make sure there is no factor sqrt(2) hiding somewhere 
    return 1. + electorn_part_1 + electorn_part_2 + ion_part
end
"
    warm_elec_dispertion

compute the warm dispersion relation for the warm electron only case
"
function warm_elec_dispertion(omega, k, V0, v_t_e, debye_e)
    z_plus  = (omega / k + V0) / (sqrt(2.) * v_t_e)
    z_minus = (omega / k - V0) / (sqrt(2.) * v_t_e)
    
    electorn_part_1 = 1.0 / ( debye_e * k )^2.0 * 1.0/2.0 * W_val(z_plus)
    electorn_part_2 = 1.0 / ( debye_e * k )^2.0 * 1.0/2.0 * W_val(z_minus)
    return 1. + electorn_part_1 + electorn_part_2
end
"
    cold_dispertion_relation

compute the cold dispersion relation for the cold electron-proton case
"
function cold_dispertion_relation(omega, k, V0, omega_e, omega_i)
    #TODO add special case for small V0*k?
    electorn_part_1 = 1.0/2.0 * omega_e^2  * ( 1 / (omega + V0 *k)^2 )
    electorn_part_2 = 1.0/2.0 * omega_e^2  * ( 1 / (omega - V0 *k)^2 )
    ion_part        =           omega_i^2        / omega^2
    return 1 - electorn_part_1 - electorn_part_2 - ion_part
end
"
    cold_dispertion_relation

compute the cold dispersion relation for the cold electron only case
"
function cold_elec_dispertion_relation(omega, k, V0, omega_e)
    #TODO add special case for small V0*k?
    electorn_part_1 = 1.0/2.0 * omega_e^2  * ( 1 / (omega + V0 *k)^2 )
    electorn_part_2 = 1.0/2.0 * omega_e^2  * ( 1 / (omega - V0 *k)^2 )
    return 1 - electorn_part_1 - electorn_part_2
end

#-------- function wrappers to be used by the newton-rhapson implementation ----------------
function function_wrapper_warm_full(omega, args...)
    return full_dispertion_relation(omega, args...)
end

function function_wrapper_warm_elec(omega, args...)
    return warm_elec_dispertion(omega, args...)
end

function function_wrapper_cold_full(omega, args...)
    return cold_dispertion_relation(omega, args...)
end

function function_wrapper_cold_elec(omega, args...)
    return cold_elec_dispertion_relation(omega, args...)
end
#----------------------------------------

In [ ]:
function numerical_derivative_imag(f, z; h=1e-5)
    h_val = h * 1.0im
    return (f(z + h_val) - f(z - h_val)) / (2.0*h_val)
end

function numerical_derivative_real(f, z; h=1e-5)
    h_val = h
    return (f(z + h_val) - f(z - h_val)) / (2.0*h_val)
end

function newton_raphson_complex(f, initial_guess; tol=1e-8, max_iterations=100, h=1e-5, step_size=1e-1)
    z = initial_guess
    for i in 1:max_iterations
        fz = f(z)
        dfz_real = numerical_derivative_real(f, z, h=h) 
        #dfz_imag = numerical_derivative_imag(f, z, h=h)  #TODO highly unstable if I take a step both in real and imaginary plane

        if abs(dfz_real) < tol
            return z, f(z), "derivative too small to continue"
            #println("Numerical derivative too small; division by zero or near-zero encountered")
        end

        #----------------- change step size to avoid too large steps ------------
        step_real = (fz / dfz_real) 
        step = step_real #+ step_imag
        step_sign = sign(step)

        if abs(step) > abs(z)
            step = step_sign * max(abs(z), 1.0) * step_size
        else
            step *= step_size
        end
        #--------–--------–--------–--------–--------–--------–--------–

        #--------– Update value of z --------–--------–--------–
        z_new = z - step
        #--------–--------–--------–--------–--------–

        # Check for convergence
        if abs(z_new - z) < tol
            return z_new, f(z_new), "Converged after $i iterations"
        end

        z = z_new
    end

    return z, f(z),  "Stopped after reaching the maximum number of iterations"
end


In [ ]:
L = 0.001188 * 1e2 # in cm

In [ ]:
2 * pi / 3.061530765382691

In [ ]:
((2.052301867492156 / 400) / c ) / 1e-4

In [ ]:
#----------- Calculate the analytical solution for the cold electron only case -------–-------–-------–-------–-------–-------–-------–
k_vals           = collect(range(0.1,stop=2, length=2000)) * omega_e / V_0  
#Chose k values such that k * V_0 / omega_e is in the range [0, 2]

cold_omega_1 = -sqrt.( (-sqrt.(8. .* k_vals.^2 .* omega_e.^2 .* V_0^2 .+ omega_e^4 .+ 0.0im) .+ 2 .* k_vals.^2 .* V_0^2 .+ omega_e^2) ./ 2 .+ 0im)
cold_omega_2 = -sqrt.( (+sqrt.(8. .* k_vals.^2 .* omega_e.^2 .* V_0^2 .+ omega_e^4 .+ 0.0im) .+ 2 .* k_vals.^2 .* V_0^2 .+ omega_e^2) ./ 2 .+ 0im)
cold_omega_3 =  sqrt.( (-sqrt.(8. .* k_vals.^2 .* omega_e.^2 .* V_0^2 .+ omega_e^4 .+ 0.0im) .+ 2 .* k_vals.^2 .* V_0^2 .+ omega_e^2) ./ 2 .+ 0im)
cold_omega_4 =  sqrt.( (+sqrt.(8. .* k_vals.^2 .* omega_e.^2 .* V_0^2 .+ omega_e^4 .+ 0.0im) .+ 2 .* k_vals.^2 .* V_0^2 .+ omega_e^2) ./ 2 .+ 0im);


println(k_vals[1], " ", k_vals[end])
println(k_vals[argmax(imag.(cold_omega_3))])
println(cold_omega_3[argmax(imag.(cold_omega_3))])
println(cold_omega_3[argmax(imag.(cold_omega_3))] / omega_e)

In [ ]:
#-------– Plot the analytical solution for the positive omega cases -------–-------–-------–-------– 
plot(k_vals * V_0 / omega_e, real.(cold_omega_3 / omega_e), label="sol1 real", title="Analytical solution to cold electron-only dispertion relation",
    size=(1000,500), xlabel="k * V_0 / omega_e", ylabel = "Ω / Ω_e")
plot!(k_vals* V_0 / omega_e, imag.(cold_omega_3/ omega_e), label="sol1 imag")
plot!(k_vals* V_0 / omega_e, real.(cold_omega_4/ omega_e), label="sol2 real")
plot!(k_vals* V_0 / omega_e, imag.(cold_omega_4/ omega_e), label="sol2 imag")

In [ ]:
cold_electron_branches = [[],[]]

for i in 1:size(k_vals,1)
    args_cold_elec = (k_vals[i], V_0, omega_e)
    res = function_wrapper_cold_elec(cold_omega_3[i], args_cold_elec...)

    if abs(res) < 1e-5
        push!(cold_electron_branches[1], ( cold_omega_3[i], k_vals[i], res))
    end 

    res = function_wrapper_cold_elec(cold_omega_4[i], args_cold_elec...)

    if abs(res) < 1e-5
        push!(cold_electron_branches[2], ( cold_omega_4[i], k_vals[i], res))
    end
end

In [ ]:
function make_branch_guesses(function_wrapper,temp_list, initial_guess, args, max_iterations=200, h=1e-5, branch_step=1e-1, reversed=false)

    k = args[1]

    # Make new initial guesses that may detect new branches
    initial_guess_1  = initial_guess + branch_step * omega_e
    initial_guess_2  = initial_guess - branch_step * omega_e
    initial_guess_3  = initial_guess + branch_step * omega_e * 1im
    initial_guess_4  = initial_guess - branch_step * omega_e * 1im

    res1, err1, msg1 = newton_raphson_complex(x -> function_wrapper(x, args...), initial_guess_1,max_iterations=max_iterations,h=h)
    res2, err2, msg2 = newton_raphson_complex(x -> function_wrapper(x, args...), initial_guess_2,max_iterations=max_iterations,h=h)
    res3, err3, msg3 = newton_raphson_complex(x -> function_wrapper(x, args...), initial_guess_3,max_iterations=max_iterations,h=h)
    res4, err4, msg4 = newton_raphson_complex(x -> function_wrapper(x, args...), initial_guess_4,max_iterations=max_iterations,h=h)
    



    if abs(err1) < 1e-5 && abs(res1 - initial_guess) > 1e-4
        push!(temp_list, (res1, k, err1))
    end
    if abs(err2) < 1e-5 && abs(res2 - initial_guess) > 1e-4
        push!(temp_list, (res2, k, err2))
    end
    if abs(err3) < 1e-5 && abs(res3 - initial_guess) > 1e-4
        push!(temp_list, (res3, k, err3))
    end
    if abs(err4) < 1e-5 && abs(res4 - initial_guess) > 1e-4
        push!(temp_list, (res4, k, err4))
    end

end 

In [ ]:
function make_new_guess(branch_list, j , function_wrapper, args, search_type=1,  max_iterations=200, h=1e-5, branch_step=1e-1, reversed=false)

    k = args[1]

    if reversed
        initial_guess = branch_list[j][1][1] 
    else
        initial_guess = branch_list[j][end][1]
    end


    #if search_type == 0
    #    initial_guess = initial_guess
    if search_type == 1
        initial_guess = initial_guess + branch_step * omega_e
    elseif search_type == 2
        initial_guess = initial_guess - branch_step * omega_e
    elseif search_type == 3
        initial_guess = initial_guess + branch_step * omega_e * 1im
    elseif search_type == 4
        initial_guess = initial_guess - branch_step * omega_e * 1im
    end



    res0, err0, msg0 = newton_raphson_complex(x -> function_wrapper(x, args...), initial_guess,max_iterations=max_iterations,h=h)

    res0 = abs(real(res0)) + imag(res0) * 1im
    #if error is still too large, do not add it to the list
    if abs(err0) < 1e-5
        if reversed
            pushfirst!(branch_list[j],  (res0, k, err0))
        else
            push!(branch_list[j],  (res0, k, err0))
        end
    end

end 

In [ ]:
function make_initial_branches(function_wrap ,arg_static, k_vals, initial_guesses, max_iterations=200, h=1e-5, k_cut=20.0, reversed=false)
    branch_list = [[] for i in 1:size(initial_guesses)[1]]
    
    k_size = size(k_vals)[1]
    if reversed
        k_cut = k_vals[end] - k_cut
    end
    cut_index = argmin(abs.(k_vals .- k_cut))


    if reversed
        for j in 1:size(initial_guesses)[1]

            i = size(initial_guesses[j],1)
            k = initial_guesses[j][i][2]

            while k > k_cut
                args = (k, arg_static...)
                initial_guess = initial_guesses[j][i][1]
                res, err, msg = newton_raphson_complex(x -> function_wrap(x, args...), initial_guess,max_iterations=max_iterations,h=h)

                if abs(err) < 1e-5
                    pushfirst!(branch_list[j],  (res, k, err))
                end

                i -= 1
                k = initial_guesses[j][i][2]
            end
        end

    else
        for j in 1:size(initial_guesses)[1]

            i = 1
            k = initial_guesses[j][i][2]

            while k < k_cut
                args = (k, arg_static...)
                initial_guess = initial_guesses[j][i][1]
                res, err, msg = newton_raphson_complex(x -> function_wrap(x, args...), initial_guess,max_iterations=max_iterations,h=h)

                if abs(err) < 1e-5
                    push!(branch_list[j],  (res, k, err))
                end

                i += 1
                k = initial_guesses[j][i][2]
            end
        end
    end

    return branch_list, cut_index
end

In [ ]:
"""
    check_new_branches

    Check if the new solutions are part of a new branch or if it is a continuation of an existing branch
    Add new branches to the branch_list if it is a new branch
"""
function check_new_branches_old(branch_list, temp_list, reversed=false)
    for (res, k, err, branch_nr) in temp_list

        new_branch = falses(size(branch_list)[1])


        for j in 1:size(branch_list)[1]

            if reversed
                k_branch = branch_list[j][1][2]
                branch_res = branch_list[j][1][1]
            else
                k_branch = branch_list[j][end][2]
                branch_res = branch_list[j][end][1]
            end

            if abs(k - k_branch) < 1e-6
                if abs(res - branch_res) > 1e-4
                    new_branch[j] = true
                end
            else
                new_branch[j] = true
            end
        end


        if all(new_branch .== true)
            if reversed
                pushfirst!(branch_list, [(res, k, err)])
            else
                push!(branch_list, [(res, k, err)])
            end 
        end


    end
end

In [ ]:
"""
    check_new_branches

    Check if the new solutions are part of a new branch or if it is a continuation of an existing branch
    Add new branches to the branch_list if it is a new branch
"""
function check_new_branches(branch_list, new_branch_list, temp_list, indices)
    for (res, k, err) in temp_list

        new_branch = falses(size(branch_list)[1])


        for j in 1:size(branch_list)[1]
            k_branch = branch_list[j][indices[j]][2]
            branch_res = branch_list[j][indices[j]][1]

            if abs(k - k_branch) < 1e-6 #This is just a sanity check - chosing the indices should make this condition always true
                if abs(res - branch_res) > 1e-4
                    new_branch[j] = true
                end
    
            end
        end


        if all(new_branch .== true)
            push!(new_branch_list, [(res, k, err)])
        end
    end



    # Now check the new_branch_list for duplicates by compariton the res values
    
    new_branch = falses(size(new_branch_list)[1])
    new_branch .= true
    for i in 1:size(new_branch_list)[1]
        for j in i+1:size(new_branch_list)[1]
            if abs(new_branch_list[i][1][1] - new_branch_list[j][1][1]) < 1e-4
                new_branch[j] = new_branch[j] && false
            end
        end
    end
    #now remove the duplicates
    new_branch_list = new_branch_list[new_branch]

end

In [ ]:
function find_matching_k(branch1, branch2, idx1, idx2, k1, k2)
    #println("input ", idx1, " ", idx2, " ", k1, " ", k2)

    new_idx1 = idx1
    new_idx2 = idx2

    new_k1 = k1
    new_k2 = k2
    while (abs(k1 - k2) > 1e-6) && idx1 < size(branch1, 1) && idx2 < size(branch2, 1)
        if k1 > k2
            idx2 += 1
            k2 = branch2[idx2][2]
        else
            idx1 += 1
            k1 = branch1[idx1][2]
        end
    end
    #println("output ", idx1, " ", idx2, " ", k1, " ", k2)
    idx1 = max(idx1, size(branch1, 1))
    idx2 = max(idx2, size(branch2, 1))
    return idx1, idx2, k1, k2
end 

In [ ]:
function find_branch_k_index(branch, k)

    if k < branch[1][2] || k > branch[end][2]
        #error("k is outside the range of the branch")   
        return -1
    end

    left = 1
    right = size(branch,1)

    while left <= right
        mid = (left + right) ÷ 2

        if abs(branch[mid][2] - k) < 1e-6
            return mid
        elseif branch[mid][2] < k
            left = mid + 1
        else
            right = mid - 1
        end
    end

    # Check which neighboring value is closest
    if abs(branch[left][2] - k) < abs(branch[right][2] - k)
        return left
    else
        return right
    end
end


In [ ]:
"""
    prune_branches

    Prunes the branches in the branch_list based on the difference in k values.
    Removes branches with large differences in k values and removes duplicate branches.

    Args:
    - branch_list: List of branches, where each branch is a list of tuples (res, k, err).
    - k_large_diff: Threshold for the difference in k values to prune branches.
    - reversed: Boolean indicating if the branches are in reverse order.

    Returns:
    - pruned_branch_list: List of pruned branches.

"""
function prune_branches(branch_list, k_large_diff, tol=1e-4)
    pruned_branch_list = []
    #First, remove all branches will large differences in k values
    for branch in branch_list

        k_values = [b[2] for b in branch]
        res_values = [b[1] for b in branch]
        k_diff = diff(k_values)
        res_diff = diff(res_values)

        #println( res_diff / k_diff )
        if any(abs.(k_diff) .> k_large_diff)# || any(real.(res_diff) .> 1e-2 * omega_e)  || any(imag.(res_diff) .> 1e-2 * omega_e)  # Check if any absolute difference in k values is larger than the threshold
            continue  # Skip this branch if it has large differences in k values
        end
        push!(pruned_branch_list, branch)
    end


    duplciate = falses((size(pruned_branch_list,1),size(pruned_branch_list,1)))

    for i in 1:size(pruned_branch_list,1)
        for j in i+1:size(pruned_branch_list,1)

            k_min1 = pruned_branch_list[i][1][2]
            k_max1 = pruned_branch_list[i][end][2]
            k_min2 = pruned_branch_list[j][1][2]
            k_max2 = pruned_branch_list[j][end][2]



            if i == j || k_max1 < k_min2 || k_max2 < k_min1
                continue
            end

            idx1 = 1; idx2 = 1
            k1 = pruned_branch_list[i][idx1][2]
            k2 = pruned_branch_list[j][idx2][2]
            diff = []
            #println(i, " ", j, " ", size(pruned_branch_list[i],1), " ", size(pruned_branch_list[j],1))
    
            while idx1 < size(pruned_branch_list[i],1)
                k1 = pruned_branch_list[i][idx1][2]

                idx1, idx2, k1, k2 = find_matching_k(pruned_branch_list[i], pruned_branch_list[j], idx1, idx2, k1, k2)    
    
                if abs(k1 - k2) < 1e-6
                    push!(diff, abs(pruned_branch_list[i][idx1][1] - pruned_branch_list[j][idx2][1]))
                end
                idx1 +=1
            end

            #println(diff)

            #println(i, " ", j, " ", maximum(diff), " ", minimum(diff))
            if maximum(diff) < tol
                duplciate[i,j] = true
            end
        end
    end

    #println(duplciate)


    final_list = []
    for i in 1:size(duplciate,1)
        if any(duplciate[:,i])
            continue
        end
        push!(final_list, branch_list[i])
    end

    return final_list
end


In [ ]:
function complete_partial_branches(function_wrap,branch_list, initial_guesses, k_vals, max_iterations=200, h=1e-5, reversed=false)

    if reversed
        for j in size(initial_guesses,1):size(branch_list,1)
            k_first = branch_list[j][end][2]
            k_first_index = argmin(abs.(k_vals .- k_first)) + 1
            for i in k_first_index:size(k_vals,1)
                args = (k_vals[i], arg_static...)
                initial_guess = branch_list[j][end][1]
                res, err, msg = newton_raphson_complex(x -> function_wrap(x, args...), initial_guess,max_iterations=max_iterations,h=h)
    
                if abs(err) < 1e-5
                    pushfirst!(branch_list[j],  (res, k_vals[i], err))
                end
            end
        end
    else
        for j in size(initial_guesses,1):size(branch_list,1)
            k_first = branch_list[j][1][2]
            k_first_index = argmin(abs.(k_vals .- k_first)) - 1
            for i in k_first_index:-1:1
                args = (k_vals[i], arg_static...)
                initial_guess = branch_list[j][1][1]
                res, err, msg = newton_raphson_complex(x -> function_wrap(x, args...), initial_guess,max_iterations=max_iterations,h=h)
    
                if abs(err) < 1e-5
                    pushfirst!(branch_list[j],  (res, k_vals[i], err))
                end
            end
        end
    end
end 

In [ ]:
function find_branch_solutions(function_wrap ,arg_static, k_vals, initial_guesses, max_iterations=200, h=1e-5, branch_step=1e-1, k_cut=20.0)
    k_size = size(k_vals)[1]

    

    combined_branch_list = []

    for search_type in 1:4
        # Search through existing branches both forwards and backwards
        if search_type == 1
            branch_list, cut_index = make_initial_branches(function_wrap ,arg_static, k_vals, initial_guesses, max_iterations, h, k_cut)
            branch_list_rev, cut_index_rev = make_initial_branches(function_wrap ,arg_static, k_vals, initial_guesses, max_iterations, h, k_cut,true)
        else
            branch_list, cut_index = make_initial_branches(function_wrap ,arg_static, k_vals, combined_branch_list, max_iterations, h, k_cut)
            branch_list_rev, cut_index_rev = make_initial_branches(function_wrap ,arg_static, k_vals, combined_branch_list, max_iterations, h, k_cut,true)   
        end

        for i in cut_index+1:k_size
            k = k_vals[i]
            args = (k, arg_static...)
            for j in 1:size(branch_list)[1]
                make_new_guess(branch_list, j, function_wrap, args, search_type, max_iterations, h, branch_step)
            end

        end

        for i in cut_index_rev-1:-1:1
            k = k_vals[i]
            args = (k, arg_static...)
            for j in 1:size(branch_list_rev)[1]
                make_new_guess(branch_list_rev, j, function_wrap, args, search_type, max_iterations, h, branch_step,true)
            end

        end

        #Combine the branches and remove dulplicates 
        combined_branch_list = vcat(combined_branch_list, branch_list)
        combined_branch_list = vcat(combined_branch_list, branch_list_rev)
        #println("search_type", search_type, " Number of branches before pruning", size(combined_branch_list,1))

        #for branch_list in combined_branch_list
        #    complete

        combined_branch_list = prune_branches(combined_branch_list, 5.0,1e-4)

        #println("search_type", search_type, " Number of branches after pruning", size(combined_branch_list,1))

    end
    
    return combined_branch_list
end


function remove_negative_branches(branches)
    new_branches = []
    for branch in branches
        if size(branch)[1] > 0
            #Check if a large area of the solution exist where the imaginary part of omega is negative and the real part is zero (or very small)
            #If so, remove the branch
            keep_branch = true
            for i in 1:size(branch)[1]
                if imag(branch[i][1]) < 1e-5 && abs.(real(branch[i][1])) < 1e-5
                    keep_branch = keep_branch && false
                end
            end

            if keep_branch
                push!(new_branches, branch)
            end
            
        end
    end
    return new_branches
end



function plot_branches(branches)
    # Initialize your plot object here, before the loop.
    # Use an empty plot as a starting point.
    p1 = plot(title="Omega vs. k", xlabel="k", ylabel="Omega / omega_e", legend=:topright)


    p2 = plot(title="Dispertion value for solution", xlabel="k", ylabel="", legend=:topright)

    for j in 1:size(branches)[1]
        omega_vals = [branches[j][i][1] for i in 1:size(branches[j])[1]]
        kvals      = [branches[j][i][2] for i in 1:size(branches[j])[1]]
        error_vals = [branches[j][i][3] for i in 1:size(branches[j])[1]]

        # Add each branch to the plot. Use unique labels for each branch for clarity.
        plot!(p1, kvals * V_0 / omega_e, real.(omega_vals) / omega_e, label="Re(omega) Branch $j")
        plot!(p1, kvals * V_0 / omega_e, imag.(omega_vals) / omega_e, label="Im(omega) Branch $j", linestyle=:dash)


        plot!(p2, kvals * V_0 / omega_e, abs.(error_vals), label="error Branch $j")

    end

    # Display the plot
    p = plot(p1,p2, size=(1400,600))
     display(p)
end 


In [ ]:
arg_static = ( V_0, omega_e)

initial_guesses = cold_electron_branches

branches_cold_elec = find_branch_solutions(function_wrapper_cold_elec, arg_static,  k_vals, initial_guesses, 200, 1e-5,1e-2,30)
#branches_cold_elec = remove_negative_branches(branches_cold_elec)
plot_branches(branches_cold_elec)


In [ ]:
#branches = [[]]
arg_static = ( V_0, omega_e, omega_p)
#initial_guesses = sol_cold1 #[sol_cold1, sol_cold2]
#initial_guesses = [sol_cold1, sol_cold2]

initial_guesses = cold_electron_branches

branches_cold_full = find_branch_solutions(function_wrapper_cold_full, arg_static,  k_vals, initial_guesses, 200, 1e-5,1e-2,30)
#branches_cold_full = remove_negative_branches(branches_cold_full)
plot_branches(branches_cold_full)

In [ ]:
arg_static = (V_0, v_t_e, debye_e)
#initial_guesses = branches_cold_full
#initial_guesses = branches_cold_full

initial_guesses = branches_cold_elec

branches_warm_elec = find_branch_solutions(function_wrapper_warm_elec, arg_static,  k_vals, initial_guesses, 200, 1e-5,1e-2,30)
plot_branches(branches_warm_elec)

In [ ]:
arg_static = (V_0, v_t_e, v_t_p, debye_e, debye_i)
initial_guesses = branches_cold_full
#initial_guesses = branches_warm_elec

branches_warm_full = find_branch_solutions(function_wrapper_warm_full, arg_static,  k_vals, initial_guesses, 300, 1e-5,4e-2,30)
plot_branches(branches_warm_full)


In [ ]:
temp_scale = 1.2

v_t_e_new = v_t_e * temp_scale
v_t_p_new =  v_t_p*temp_scale
debye_e_new = v_t_e_new / omega_e
debye_i_new = v_t_p_new / omega_p

arg_static = (V_0 , v_t_e_new, v_t_p_new, debye_e_new, debye_i_new)
#initial_guesses = branches_cold_full
#initial_guesses = branches_warm_full[1:3]
initial_guesses = branches_warm_full

branches_warm_full_v02 = find_branch_solutions(function_wrapper_warm_full, arg_static,  k_vals, initial_guesses, 300, 1e-5,5e-2,40)
plot_branches(branches_warm_full_v02)
